In [ ]:
import sys
import wget
import newspaper
from newspaper import Article
import json
import pandas as pd
pd.set_option("max_colwidth", 600)
import ast
from bs4 import BeautifulSoup
import re
import requests
import time
import numpy as np
import zipfile
import os
import html
import re
import itertools
import lxml
import xmltodict
import collections
from urllib import request
from collections import OrderedDict
from urllib.error import HTTPError, URLError
from urllib.parse import urlparse
import sys
import ast
import time
from pandarallel import pandarallel
import requests
import datefinder
from datetime import datetime
from tqdm import tqdm
import psutil
import chromedriver_binary
from selenium import webdriver
from multiprocessing import Pool

In [ ]:
#PLEASE EDIT THIS CELL!

#Universal parameters. Always needs to be changed
news_outlet = "critica"
country = "panama"
max_click_SHOW_MORE = 500
host = "https://www.critica.com.pa/"
date_format = "%Y-%m-%d"
relevant_path = f'^{host}.*$'
urls = [host + 'nacional?',host + 'mundo?']
csv_name = country+'_'+news_outlet+'.csv'
types = ['page', 'click_more']
type_of_page = types[1]






# If page type is click_more i.e. can get more article by clicking a button with text 'click more'  
xpath_for_link = "//h2[contains(@class, 'titulo')]//following::a"
xpath_for_click_more = "//div[contains(@class, 'btn vermasnoticias cf')]"





# If page type is page
target_tag = 'h5'
target_tag_class = ''

if type_of_page == 'page':
    urls = [url + 'page/' for url in urls]
pages_each_category = [150,20] #The total number of pages for each section. So if policiaca is 1..92, write 92, not 93.
if pages_each_category:
    total_pages = pages_each_category
else:
    total_pages = 500

In [ ]:
def parallelize_dataframe(df, func, n_cores):
    df_split = np.array_split(df, n_cores)
    pool = Pool(n_cores)
    df = pd.concat(pool.map(func, df_split))
    pool.close()
    pool.join()
    return df

In [ ]:
def retrieve_links_from_list(url):
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome(executable_path="chromedriver", options=chrome_options, )
  driver.get(url)
  
  count = 0

  news_urls = set()

  prev_length = 0
  while count <= max_click_SHOW_MORE:
      time.sleep(1)
      try:
        element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, xpath_for_click_more))
        )
      except TimeoutException as e:
        break
        
    
      try:
          all_links = driver.find_elements(By.XPATH, xpath_for_link)
          click_more = driver.find_element(By.XPATH, xpath_for_click_more)
          curr_length = len(all_links)
          print(f'{curr_length} links in current page')
          print(f'{prev_length} links in previous page')
          if count > 0:
            if curr_length == prev_length or not click_more:
              break
          
          prev_length = curr_length

          print("Button clicked #", count)

          click_more.click();
          count += 1
          
      except TimeoutException:
          break

  time.sleep(2)

  all_links = driver.find_elements(By.XPATH, xpath_for_link)

  for link in all_links:
        if isinstance(link.get_attribute('href'), str):
          if re.match(relevant_path, link.get_attribute('href')):
            news_urls.add(link.get_attribute('href'))

  driver.quit()
  print('*' * 20)
  return list(news_urls)


def extract_urls(url):    
    rows = set()
    soup = getSoup(url)
    if soup:
      all_elems = None
      if target_tag_class != '':
        all_elems = soup.find_all(target_tag, {'class':target_tag_class})
      else:
        all_elems = soup.find_all(target_tag)
      
      for d in all_elems:
        all_links = d.find_all('a', href=True)
        
        for l in all_links:
            if not re.match('^http.*', l['href']):
              l['href'] = host + l['href']
              
            if re.match(relevant_path, l['href']):
              rows.add(l['href'])
                
    
    if rows:
      return list(rows)
    else:
      return None

def extract_urls_from_df(df):
    links = df['page'].map(lambda x: extract_urls(x))
    df['link'] = links
    return df

In [ ]:
pages = []

def get_zipped_urls(url, total_pages):
    page_numbers = [str(i) for i in range(1,total_pages)]
    url_multiple = [url] * total_pages
    return [''.join(x) for x in zip(url_multiple,page_numbers)]

if type_of_page == 'page':
    for ind, url in enumerate(urls):
        if isinstance(total_pages, int):
            pages += get_zipped_urls(url, total_pages)
        else:
            pages_for_category = total_pages[ind] + 1
            pages += get_zipped_urls(url, pages_for_category)
            
elif type_of_page == 'click_more':
    for url in urls:
        pages += retrieve_links_from_list(url)

        
df = pd.DataFrame(pages, columns = ['page'])

In [ ]:
def getSoup(url):
    """
    Return a soup object of the URL
    """
    try:
        time.sleep(5)
        req = request.Request(url, headers={'User-Agent' : "Chrome"}) 
        con = request.urlopen(req)
        html = con.read()

    except HTTPError as e:
        print(e)
        return None
        
    except URLError as e:
        print('The server could not be found')
        return None

    except Exception as e:
      print(e)
      return None

    
    soup = BeautifulSoup(html, 'html.parser')
    return soup

In [ ]:
def extract_text(url):
    text, title, date = None, None, None
    try:
        result = [country, news_outlet] + [''] * 3 
        article = Article(url, keep_article_html=False)
        article.download()
        article.parse()
        text = article.text
        
        if text:
            text_copy = text
            title = article.title
            date = article.publish_date
            if date:
                date = date.strftime(date_format)
            else:
                matches = datefinder.find_dates(text_copy)
                most_recent_datetime = sorted(matches)[-1]
                date = most_recent_datetime.strftime(date_format)
    finally:
      if title:
        result[2] = title
      if date:
        result[3] = date
      if text:
        result[4] = text
      return result 


def extract_text_from_df(df):
    content = df['link'].map(lambda x: extract_text(x))
    df['content'] = content
    return df

In [ ]:
df

,page
0,https://www.critica.com.pa/nacional?page/1
1,https://www.critica.com.pa/nacional?page/2
2,https://www.critica.com.pa/nacional?page/3
3,https://www.critica.com.pa/nacional?page/4
4,https://www.critica.com.pa/nacional?page/5
...,...
165,https://www.critica.com.pa/mundo?page/16
166,https://www.critica.com.pa/mundo?page/17
167,https://www.critica.com.pa/mundo?page/18
168,https://www.critica.com.pa/mundo?page/19


In [ ]:
def parallel_work(df, method_to_run, target, source):
  global workers
  while workers>0:
    try:
      df[target] = df[source].parallel_apply(method_to_run)
    except Exception as e:  
      raise e
    break

  if workers == 0:
    print('Error during parallel operation. Could not extract text')

  return df

def get_parallel_operation_results(divided_dfs, method_to_run, target, source):
  res = []
  for df in divided_dfs:
    try:
      temp_df = parallel_work(df, method_to_run, target, source)
      if temp_df[target].isnull().all():
        print('Could not retrieve any URLS')
        print('Something is wrong with the target_tag and target_tag_class variables. Please modify')
        return []
      res.append(temp_df)
    except Exception:
      continue
  df_result = pd.concat(res)
  return df_result

def partition_df(df):
  global articles_per_parallel_operation
  divided_dfs = []
  start = 0
  while start < len(df):
    divided_dfs.append(df[start:start+articles_per_parallel_operation])
    start += articles_per_parallel_operation
  return divided_dfs

In [ ]:
if type_of_page == 'page':    
    start = 0
    limit = psutil.cpu_count()
    total_time_start = time.time()
    results = []
    
    while start < len(df):
        start_time = time.time()
        results.append(parallelize_dataframe(df[start:start+limit], extract_urls_from_df, psutil.cpu_count()))
        end_time = time.time()
        print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
        print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
        start+=limit
    
    df = pd.concat(results)
    total_time_end = time.time()
    print(f'total time taken: {round(total_time_end - total_time_start,2)} second')

else:
    df = df.rename(columns={"page": "link"})

Batch of data of row range 0-96 complete in 18.78 seconds
56.47% complete
Batch of data of row range 96-192 complete in 15.99 seconds
100% complete
total time taken: 34.77 second


In [ ]:
df

,page,link
0,https://www.critica.com.pa/nacional?page/1,"[https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141, https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143, https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123, https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146, https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131, https://www.critica.com.pa//nacional/traslado-gratuito-para-viajeros-afectados-por..."
1,https://www.critica.com.pa/nacional?page/2,"[https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141, https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143, https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123, https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146, https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131, https://www.critica.com.pa//nacional/traslado-gratuito-para-viajeros-afectados-por..."
2,https://www.critica.com.pa/nacional?page/3,"[https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141, https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143, https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123, https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146, https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131, https://www.critica.com.pa//nacional/traslado-gratuito-para-viajeros-afectados-por..."
3,https://www.critica.com.pa/nacional?page/4,"[https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141, https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143, https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123, https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146, https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131, https://www.critica.com.pa//nacional/traslado-gratuito-para-viajeros-afectados-por..."
4,https://www.critica.com.pa/nacional?page/5,"[https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141, https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143, https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123, https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146, https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131, https://www.critica.com.pa//nacional/traslado-gratuito-para-viajeros-afectados-por..."
...,...,...
165,https://www.critica.com.pa/mundo?page/16,"[https://www.critica.com.pa//mundo/dimisiones-e-incertidumbre-politica-en-sri-lanka-632784, https://www.critica.com.pa//mundo/el-turismo-da-signos-de-recuperacion-en-america-tras-el-covid19-633043, https://www.critica.com.pa//mundo/murio-ivana-trump-primera-esposa-de-donald-trump-632980, https://www.critica.com.pa//mundo/tres-heridos-por-encierro-de-toros-en-san-fermin-632720, https://www.critica.com.pa//mundo/detienen-rafael-caro-quintero-el-capo-mas-buscado-por-eeuu-633019, https://www.critica.com.pa//mundo/verguero-en-sri-lanka-se-toman-casa-del-presi-632755, https://www.critica.com.pa/..."
166,https://www.critica.com.pa/mundo?page/17,"[https://www.critica.com.pa//mundo/dimisiones-e-i

In [ ]:
print(len(df))
df = df[df['link'].notna()]
df = df[(df['link'].str.len() != 0)]
print(len(df))

170
170


In [ ]:
if type_of_page == 'page':
    df = df.explode('link')
    
df

,page,link
0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141
0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143
0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123
0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146
0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131
...,...,...
169,https://www.critica.com.pa/mundo?page/20,https://www.critica.com.pa//mundo/biden-pide-que-le-manden-historias-sobre-la-violencia-y-tiroteos-632724
169,https://www.critica.com.pa/mundo?page/20,https://www.critica.com.pa//mundo/mas-de-20-muertos-por-brote-de-encefalitis-japonesa-en-la-india-633047
169,https://www.critica.com.pa/mundo?page/20,https://www.critica.com.pa//mundo/policia-y-un-civil-mueren-tras-un-ataque-con-explosivos-en-colombia-633056
169,https://www.critica.com.pa/mundo?page/20,https://www.critica.com.pa//mundo/nicaragua-extiende-el-congelamiento-del-precio-de-la-gasolina-633046


In [ ]:
print(len(df))
df = df.drop_duplicates(subset='link')
print(len(df))

2550
30


In [ ]:
output_path= 'unprocessed_' + csv_name
start = 0
limit = psutil.cpu_count(logical=False)

while start < len(df):
    start_time = time.time()
    demo_df = df[start:start+limit].copy()
    test_df = parallelize_dataframe(demo_df, extract_text_from_df, psutil.cpu_count(logical=False))
    test_df.to_csv(output_path, mode='a', header=not os.path.exists(output_path))
    end_time = time.time()
    print(f'Batch of data of row range {start}-{start+limit} complete in {round(end_time-start_time, 2)} seconds')
    print(f'{round(min((((start+limit) / len(df)) * 100), 100), 2)}% complete')
    start+=limit


Batch of data of row range 0-48 complete in 2.44 seconds
160.0% complete


In [ ]:
output_path= 'unprocessed_' + csv_name
if 'df' not in locals() or df is None or 'content' not in df:
    if(os.path.exists(output_path)):
        df = pd.read_csv(output_path, converters={'content': pd.eval})



In [ ]:
df[['country', 'news_outlet', 'title', 'date', 'text']] = pd.DataFrame(df.content.tolist(), index= df.index)     
df

,Unnamed: 0,page,link,content,country,news_outlet,title,date,text
0,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141,"[panama, critica, Suspenden audiencia preliminar del caso Odebrecht [Video], 2022-07-18, Panamá- La audiencia preliminar por el caso Odebrecht fue suspendida y reprogramada para el próximo 12 de septiembre, así lo informó la juez Tercera Liquidadora de Causas Penales, Baloisa Marquínez.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nLa suspensión del acto de audiencia se debió a que algunos abogados particulares presentaron certificados de incapacidad.\n\nAdemás, algunos de los imputados en el presente proceso renunciaron a los abogados de oficios asignados y decidieron contar ...",panama,critica,Suspenden audiencia preliminar del caso Odebrecht [Video],2022-07-18,"Panamá- La audiencia preliminar por el caso Odebrecht fue suspendida y reprogramada para el próximo 12 de septiembre, así lo informó la juez Tercera Liquidadora de Causas Penales, Baloisa Marquínez.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nLa suspensión del acto de audiencia se debió a que algunos abogados particulares presentaron certificados de incapacidad.\n\nAdemás, algunos de los imputados en el presente proceso renunciaron a los abogados de oficios asignados y decidieron contar con defensa particular, por lo que se aceptó la fecha alterna para celebrar el acto.\n\nH..."
1,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143,"[panama, critica, Iglesia continúa con esfuerzos para una sola mesa de diálogo [Video], 2022-07-18, Panamá- Monseñor José Domingo Ulloa hizo un llamado a los dirigentes de las diferentes agrupaciones, a realizar una sola mesa y abordar los distintos temas por los que realizan las protestas en el país.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nExplicó que el equipo facilitador de la Iglesia Católica continúan con los esfuerzos para lograr el acercamiento entre las partes en conflicto, y que la población pueda recibir respuestas a sus peticiones sociales y económicas.\n\n\n\...",panama,critica,Iglesia continúa con esfuerzos para una sola mesa de diálogo [Video],2022-07-18,"Panamá- Monseñor José Domingo Ulloa hizo un llamado a los dirigentes de las diferentes agrupaciones, a realizar una sola mesa y abordar los distintos temas por los que realizan las protestas en el país.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nExplicó que el equipo facilitador de la Iglesia Católica continúan con los esfuerzos para lograr el acercamiento entre las partes en conflicto, y que la población pueda recibir respuestas a sus peticiones sociales y económicas.\n\n\n\nReiteró que mantendrán los esfuerzos para un acercamiento a una mesa única durante en el día de hoy..."
2,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123,"[panama, critica, Trasladan material para potabilizadoras de Veraguas y Bocas, 2022-07-18, Unidades del Servicio Nacional Aeronaval (SENAN) trasladaron en en la embarcación Senan L-402 General Esteban Huertas, hacia la Costa Norte de Veraguas y a la provincia de Bocas del Toro, 45 mil libras de sulfato de aluminio para mantener la producción de agua en las cinco potabilizadoras que atienden el suministro para la población que vive en este sector del país.\n\nComo parte de esta acción social se transportó 15 mil libras de alimentos secos y otros artículos de primera necesidad al igual 15 mi...",panama,critica,Trasladan material para potabilizadoras de Veraguas y Bocas,2022-07-18,"Unidades del Servicio Nacional Aeronaval (SENAN) trasladaron en en la embarcación Senan L-402 General Esteban Huertas, hacia la Costa Norte de Veraguas y a la provincia de Bocas del Toro, 45 mil libras 

In [ ]:
print(len(df))
df = df[df['text'] != '']
print(len(df))
df

30
30


,Unnamed: 0,page,link,content,country,news_outlet,title,date,text
0,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141,"[panama, critica, Suspenden audiencia preliminar del caso Odebrecht [Video], 2022-07-18, Panamá- La audiencia preliminar por el caso Odebrecht fue suspendida y reprogramada para el próximo 12 de septiembre, así lo informó la juez Tercera Liquidadora de Causas Penales, Baloisa Marquínez.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nLa suspensión del acto de audiencia se debió a que algunos abogados particulares presentaron certificados de incapacidad.\n\nAdemás, algunos de los imputados en el presente proceso renunciaron a los abogados de oficios asignados y decidieron contar ...",panama,critica,Suspenden audiencia preliminar del caso Odebrecht [Video],2022-07-18,"Panamá- La audiencia preliminar por el caso Odebrecht fue suspendida y reprogramada para el próximo 12 de septiembre, así lo informó la juez Tercera Liquidadora de Causas Penales, Baloisa Marquínez.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nLa suspensión del acto de audiencia se debió a que algunos abogados particulares presentaron certificados de incapacidad.\n\nAdemás, algunos de los imputados en el presente proceso renunciaron a los abogados de oficios asignados y decidieron contar con defensa particular, por lo que se aceptó la fecha alterna para celebrar el acto.\n\nH..."
1,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143,"[panama, critica, Iglesia continúa con esfuerzos para una sola mesa de diálogo [Video], 2022-07-18, Panamá- Monseñor José Domingo Ulloa hizo un llamado a los dirigentes de las diferentes agrupaciones, a realizar una sola mesa y abordar los distintos temas por los que realizan las protestas en el país.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nExplicó que el equipo facilitador de la Iglesia Católica continúan con los esfuerzos para lograr el acercamiento entre las partes en conflicto, y que la población pueda recibir respuestas a sus peticiones sociales y económicas.\n\n\n\...",panama,critica,Iglesia continúa con esfuerzos para una sola mesa de diálogo [Video],2022-07-18,"Panamá- Monseñor José Domingo Ulloa hizo un llamado a los dirigentes de las diferentes agrupaciones, a realizar una sola mesa y abordar los distintos temas por los que realizan las protestas en el país.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nExplicó que el equipo facilitador de la Iglesia Católica continúan con los esfuerzos para lograr el acercamiento entre las partes en conflicto, y que la población pueda recibir respuestas a sus peticiones sociales y económicas.\n\n\n\nReiteró que mantendrán los esfuerzos para un acercamiento a una mesa única durante en el día de hoy..."
2,0,https://www.critica.com.pa/nacional?page/1,https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123,"[panama, critica, Trasladan material para potabilizadoras de Veraguas y Bocas, 2022-07-18, Unidades del Servicio Nacional Aeronaval (SENAN) trasladaron en en la embarcación Senan L-402 General Esteban Huertas, hacia la Costa Norte de Veraguas y a la provincia de Bocas del Toro, 45 mil libras de sulfato de aluminio para mantener la producción de agua en las cinco potabilizadoras que atienden el suministro para la población que vive en este sector del país.\n\nComo parte de esta acción social se transportó 15 mil libras de alimentos secos y otros artículos de primera necesidad al igual 15 mi...",panama,critica,Trasladan material para potabilizadoras de Veraguas y Bocas,2022-07-18,"Unidades del Servicio Nacional Aeronaval (SENAN) trasladaron en en la embarcación Senan L-402 General Esteban Huertas, hacia la Costa Norte de Veraguas y a la provincia de Bocas del Toro, 45 mil libras 

In [ ]:
if 'page' in df:
  df = df.drop(['page'], axis=1)
if 'content' in df:
  df = df.drop(['content'], axis=1)
if 'Unnamed: 0' in df:
    df = df.drop(['Unnamed: 0'], axis=1)

In [ ]:
df

,link,country,news_outlet,title,date,text
0,https://www.critica.com.pa//nacional/suspenden-audiencia-preliminar-del-caso-odebrecht-video-633141,panama,critica,Suspenden audiencia preliminar del caso Odebrecht [Video],2022-07-18,"Panamá- La audiencia preliminar por el caso Odebrecht fue suspendida y reprogramada para el próximo 12 de septiembre, así lo informó la juez Tercera Liquidadora de Causas Penales, Baloisa Marquínez.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nLa suspensión del acto de audiencia se debió a que algunos abogados particulares presentaron certificados de incapacidad.\n\nAdemás, algunos de los imputados en el presente proceso renunciaron a los abogados de oficios asignados y decidieron contar con defensa particular, por lo que se aceptó la fecha alterna para celebrar el acto.\n\nH..."
1,https://www.critica.com.pa//nacional/iglesia-continua-con-esfuerzos-para-una-sola-mesa-de-dialogo-video-633143,panama,critica,Iglesia continúa con esfuerzos para una sola mesa de diálogo [Video],2022-07-18,"Panamá- Monseñor José Domingo Ulloa hizo un llamado a los dirigentes de las diferentes agrupaciones, a realizar una sola mesa y abordar los distintos temas por los que realizan las protestas en el país.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\nExplicó que el equipo facilitador de la Iglesia Católica continúan con los esfuerzos para lograr el acercamiento entre las partes en conflicto, y que la población pueda recibir respuestas a sus peticiones sociales y económicas.\n\n\n\nReiteró que mantendrán los esfuerzos para un acercamiento a una mesa única durante en el día de hoy..."
2,https://www.critica.com.pa//nacional/trasladan-material-para-potabilizadoras-de-veraguas-y-bocas-633123,panama,critica,Trasladan material para potabilizadoras de Veraguas y Bocas,2022-07-18,"Unidades del Servicio Nacional Aeronaval (SENAN) trasladaron en en la embarcación Senan L-402 General Esteban Huertas, hacia la Costa Norte de Veraguas y a la provincia de Bocas del Toro, 45 mil libras de sulfato de aluminio para mantener la producción de agua en las cinco potabilizadoras que atienden el suministro para la población que vive en este sector del país.\n\nComo parte de esta acción social se transportó 15 mil libras de alimentos secos y otros artículos de primera necesidad al igual 15 mil libras de medicamentos distribuidas en varias cajas del Ministerio de Salud y que serán e..."
3,https://www.critica.com.pa//nacional/montilla-anadepo-tiene-agenda-oculta-para-desestabilizar-gobierno-633146,panama,critica,Montilla: Anadepo tiene agenda oculta para desestabilizar a Gobierno,2022-07-18,"Panamá- ""El Gobierno Nacional va a cumplir con lo pactado, y hoy a partir de las 3:00 p.m. todos van a poder ir a las estaciones de combustible a echar gasolina al precio de $3.25 el galón"", afirmó Omar Montilla, viceministro de Comercio Interior e Industrias.\n\nLEE TAMBIÉN: LAS ÚLTIMAS NOTICIAS AQUÍ\n\nPublicidad\n\n\n\nMontilla tachó como lamentable que Anadepo no haya cumplido con lo pactado.\n\nEl funcionario ""lamentó que hoy se amanezca con una ruptura del acuerdo de parte del señor Luis Sánchez, de la Asociación de Educadores Veragüenses (AEVE), porque hay que aclarar que las otras ..."
4,https://www.critica.com.pa//nacional/no-es-puja-y-repuja-es-hartazgo-social-633131,panama,critica,"¡No es puja y repuja, es hartazgo social!",2022-07-18,"¡No se trata de puja y repuja por centésimos en el precio del galón del combustible!, se trata de decisiones políticas-económicas que hagan que el pueblo pueda recuperar su poder adquisitivo, es el criterio de la alianza Pueblo Unido por la Vida.\n\nEl grupo insistió ayer en establecer una sola mesa de diálogo con el Gobierno para que responda a diversos reclamos de equidad y justicia social, porque lo medular no son centésimos más o menos en la rebaja del combustible, sino que lo medular es la desigualdad Saúl Méndez insistió que en Panamá el pueblo no se ha levantado por centavos de la g..

In [ ]:
df.to_csv(csv_name)

In [ ]:
if(os.path.exists(output_path)):
    os.remove(output_path)
    print(('unprocessed file deleted'))

unprocessed file deleted
